In [ ]:
import requests
import sys
import json
from dotenv import load_dotenv
import os
import re
import pandas as pd
from functools import wraps
import time
from numpy import dtype
from sshtunnel import SSHTunnelForwarder
import psycopg2
from datetime import datetime
from sqlalchemy import create_engine
import hashlib
import undetected_chromedriver as uc
from webdriver_manager.chrome import ChromeDriverManager
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import re
import pandas as pd
import threading
from multiprocessing import Pool
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import logging

logging.basicConfig(filename="fundatmentals_runner.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

load_dotenv()

api_key=os.getenv('API_KEY')
DB_ENV_PROD=1

def send_mail(send_from: str,subject: str,text: str,send_to: list,files=None):
    send_to = default_address if not send_to else send_to
    msg = MIMEMultipart()
    username = 'faghost6201@gmail.com'
    password = 'mochdbbyiiwmxzzg'
    msg['From'] = send_from
    msg['To'] = ', '.join(send_to)
    msg['Subject'] = subject
    part1 = MIMEText(text, 'plain')
    msg.attach(part1)
    for f in files or []:
        with open(f, "rb") as fil:
            ext = f.split('.')[-1:]
            attachedfile = MIMEApplication(fil.read(), _subtype=ext)
            attachedfile.add_header('content-disposition',
                                    'attachment',
                                    filename=basename(f))
        msg.attach(attachedfile)

    if '@gmail' in send_from:
        smtp = smtplib.SMTP(host="smtp.gmail.com", port=587)
    elif '@yahoo' in send_from:
        smtp = smtplib.SMTP(host="smtp.mail.yahoo.com", port=587)
    elif '@outlook' in send_from or '@hotmail' in send_from:
        smtp = smtplib.SMTP(host="smtp-mail.outlook.com", port=587)
    elif '@verizonwireless' in send_from:
        smtp = smtplib.SMTP(host="smtp.verizon.net", port=465)

    smtp.starttls()
    smtp.login(username, password)
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.close()

def replace_values(list_to_replace, item_to_replace, item_to_replace_with):
    return [item_to_replace_with if item == item_to_replace else item for item in list_to_replace]



response_stock_list=requests.get(f'https://fmpcloud.io/api/v3/stock/list?apikey={api_key}')
data_stock_list=response_stock_list.json()

required_companies_symbol=[]
for company in data_stock_list:
    if company["exchangeShortName"]=='NYSE' or company["exchangeShortName"]=='NASDAQ':
        required_companies_symbol.append(company['symbol'])
if DB_ENV_PROD == 1:
    conn_params = {
    'database': os.environ.get('CLOUD_DB_NAME'), 
    'user': os.environ.get('CLOUD_DB_USER'), 
    'password': os.environ.get('CLOUD_DB_PASSWORD'), 
    'host': os.environ.get('CLOUD_DB_HOST'), 
    'port': int(os.environ.get('CLOUD_DB_PORT'))
    }
else:
    conn_params = {
    'database': os.environ.get('LOCAL_DB_NAME'), 
    'user': os.environ.get('LOCAL_DB_USER'), 
    'password': os.environ.get('LOCAL_DB_PASSWORD'), 
    'host': os.environ.get('DB_HOST'), 
    'port': int(os.environ.get('DB_PORT'))
    }

REMOTE_HOST = os.environ.get('REMOTE_HOST')
REMOTE_USERNAME = os.environ.get('REMOTE_USERNAME')
# PKEY_PATH= os.environ.get('PKEY_PATH')

def get_value(sql, port=5432):
    """
    pass query in parameter 'sql' to return result from table in db 
    """
    try:
        conn_params['port'] = port
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        cur.execute(sql)
        val = cur.fetchall()
        conn.close()
        return val
    
    except Exception as e:
        logger.error(e)


# set value
def set_value(sql, port=5432):
    """
    pass query in parameter 'sql' to commit result from table in db 
    """
    try:
        conn_params['port'] = port
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        conn.close()
    
    except Exception as e:
        logger.error(e)

# def open_ssh_tunnel(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         if DB_ENV_PROD:

#             tunnel = SSHTunnelForwarder((REMOTE_HOST),
#                 ssh_pkey=PKEY_PATH,
#                 ssh_username=REMOTE_USERNAME,
#                 remote_bind_address=(conn_params['host'],int(conn_params['port'])),
#                 )
#             tunnel.start()
#             conn_params['port'] = tunnel.local_bind_port

#         result = func(*args, **kwargs)
        
#         if DB_ENV_PROD: tunnel.stop()
#         return result
#     return wrapper

# @open_ssh_tunnel
def test_func():
    conn = psycopg2.connect(**conn_params)
    cur = conn.cursor()
    cur.execute("SELECT count(*) FROM earning_calendar LIMIT 1")
    print(cur.fetchall())
    conn.close()
    
def get_symbols_exchanges(API_KEY, table_name, port=5432):
    '''
    To pull the symbols and exchange data via the fmpcloud api endpoint (stock).
    
    :param url: String url to fetch data from
    '''
    logger.info('Getting Symbol Exchanges!')
    try:
        print('fetching symbols and exchanges data')
        url = "https://fmpcloud.io/api/v3/stock/list?apikey=%s"%API_KEY
        session = create_session()
        data = get_api(session, url)
        if data:
            df = pd.DataFrame(data)
            # get only symbols from 'NASDAQ','NYSE' exchanges
            df = df.set_index('exchangeShortName').loc[['NASDAQ','NYSE'],'symbol'].reset_index()
            if table_name:
                try:
                    # order symbols based on values from existing table;
                    # i.e create a list of symbols that don't exist in database first, 
                    # and then append the remaining symbols to that list
                    values = get_value('SELECT distinct %s from %s'%('symbol',table_name), port=port)
                    model_symbol_set = set([val[0] for val in values])
                    sheet_symbol_set = set(df['symbol'])
                    symbol_to_save = list(sheet_symbol_set - model_symbol_set)
                    # symbol_to_save.extend(model_symbol_set)
                    # get new records from df
                    df = df.set_index('symbol').loc[symbol_to_save,'exchangeShortName'].reset_index()
                except:
                    print("table %s doesn't exist; fetching all data"%table_name)

            logger.info('fetched symbols and exchanges for %s companies'%len(df))
            return df['symbol'].values, df['exchangeShortName'].values
    
    except Exception as e:
        logger.error(e)
        
def create_session():
    """
    create a requests Sesssion that defines paramerters 
    like retry count, backoff time ({backoff factor} * (2 ** ({number of total retries} - 1)))
    """
    logger.info('Creating Session')
    try:
        session = requests.Session()
        retry = Retry(total=5, connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('https://', adapter)
        return session
    except Exception as e:
        logger.error(e)

def get_api(session, url):
    """
    get call based on session created
    return data in json format, if status code is 200
    """        
    logger.info('Getting results from API')
    try:
        r = session.get(url, headers={'Content-Type': 'application/json'})
        if r.status_code == requests.codes.ok: 
            return r.json()
    except Exception as e:
        logger.error(e)
        
def create_table(values, table_name, pk='tsid', port=5432):
    logger.info('Creating Table!')
    try:
        # print('port: %s'%port)
        df = pd.DataFrame(values)
        conn_params['port'] = port
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        s = "CREATE TABLE IF NOT EXISTS %s ("%table_name
        for k,v in df.dtypes.items():
            if k == 'date':
                v = 'DATE'
            elif k in {'filling_date','accepted_date'}:
                v = 'TIMESTAMP WITH TIME ZONE'
                # v = 'TIMESTAMP WITHOUT TIME ZONE'
            elif k == 'tsid':
                v = 'VARCHAR (30)'
            elif k in {'symbol','exchange_short_name'}:
                v = 'VARCHAR (10)'
            elif k == 'calendar_year':
                v = 'INTEGER'
            elif v in {dtype('int64'), dtype('float64')}:
                # v = 'DECIMAL(12,6)'
                v = 'NUMERIC'
            elif v in {dtype('O')}:
                v = 'TEXT'
            else:
                v = 'TEXT'
            s+=f'"{k}" {v}, '
        s+='created_at TIMESTAMP WITH TIME ZONE, PRIMARY KEY(%s))'%pk
        # s+='created_at TIMESTAMP WITHOUT TIME ZONE, PRIMARY KEY(%s))'%pk
        cur.execute(s)
        conn.commit()
        conn.close()

    except Exception as e:
        logger.error(e)
def save(values, table_name, symbol=None, pk='tsid', port=5432):
    try:
        # filter for new values
        df = pd.DataFrame(values)
        conn_params['port'] = port
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        if symbol==None: cur.execute('SELECT %s from %s'%(pk,table_name))
        else: cur.execute("SELECT %s from %s where symbol = '%s'"%(pk,table_name,symbol))
        
        model_pk_set = set([val[0] for val in cur.fetchall()])
        conn.close()
        sheet_pk_set = set(df[pk])
        pk_to_save = sheet_pk_set - model_pk_set

        # get new records from df
        l = [df[df.loc[:,pk] == val] for val in pk_to_save]
        if l:
            df = pd.concat(l, ignore_index=True)
            df['created_at'] = datetime.now()

            # save new values only to table
            engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{database}".format(**conn_params))
            conn = engine.connect()
            df.to_sql(table_name, conn, if_exists='append', index=False)
            conn.close()
            logger.info("saved %s new values for %s"%(len(df), table_name))
        else:
            logger.info("no new values to save for %s"%table_name)
    
    except Exception as e:
        logger.error(e)

class FundamentalTables():

    def __init__(self, API_KEY, session, end_point, symbol, exchange, currency, country, period, limit):
        self.API_KEY, self.session, self.end_point, self.symbol, self.exchange, self.currency, self.country, self.period, self.limit = API_KEY, session, end_point, symbol, exchange, currency, country, period, limit

    def fetch_data(self):
        try:
            logger.info('fetching data from %s for %s symbol for period %s'%(self.end_point, self.symbol, self.period))
            url = f"https://fmpcloud.io/api/v3/{self.end_point}/{self.symbol}?period={self.period}&limit={self.limit}&apikey={self.API_KEY}"
            data = get_api(self.session, url)
            if data:
                df = pd.DataFrame(data)
                # add 'symbol' if not exists
                if 'symbol' not in df.columns:
                    df.insert(loc=0, column='symbol', value=self.symbol)
                # add 'exchangeShortName' if not exists
                if 'exchangeShortName' not in df.columns:
                    df.insert(2, 'exchangeShortName', self.exchange)
                # add 'currency' if not exists
                if 'currency' not in df.columns:
                    df.insert(2, 'currency', self.currency)
                # add 'country' if not exists
                if 'country' not in df.columns:
                    df.insert(2, 'country', self.country)
                # add 'calendarYear' if not exists
                if 'calendarYear' not in df.columns and self.end_point not in {'key-metrics-ttm','ratios-ttm','profile'}:
                    df.insert(3, 'calendarYear', df['date'].map(lambda x:x.split('-')[0]))
                # add 'TSID'
                if self.end_point in {'key-metrics-ttm','ratios-ttm','profile'}:
                    df.insert(0, 'TSID', df[['symbol','exchangeShortName']].apply(lambda x:'.'.join(x), axis=1))
                    count = len(df)
                elif self.period == 'quarter':
                    df.insert(1, 'TSID', df[['symbol','exchangeShortName','calendarYear','period']].apply(lambda x:'.'.join(x), axis=1))
                    count = int(len(df)/4)
                else:
                    df.insert(1, 'TSID', df[['symbol','exchangeShortName','calendarYear']].apply(lambda x:'.'.join(x), axis=1))
                    count = len(df)
                # convert column names to snake_case
                df.rename(columns={col:self.camel_to_snake(col) for col in df.columns}, inplace=True)
                
                logger.info('Receieved %s years of data for company %s'%(count, self.symbol))
                return df.to_dict('records')
        
        except Exception as e:
            logger.error(e)

    def camel_to_snake(self, name):
        name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
        return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()




def main(*args):
    """
    *args -> (('income-statement', 'annual', 30, 'annual_income_statement'),)
    end_point, period, limit, table_name = 'income-statement', 'annual', 30, 'annual_income_statement'
    """
    try:
        end_point, period, limit, table_name, port = args[0]
        logger.info('port: %s'%port)
        # get symbols and exchange data
        symbols, exchanges = get_symbols_exchanges(api_key, table_name, port=port)
        
        # alter table to add new columns
        set_value("""
            alter table if exists %s 
            add column if not exists country varchar(30), 
            add column if not exists currency varchar(10)
        """%table_name, port=port)
        session = create_session()
        print('fetching data from %s for %s companies for period %s'%(end_point, len(symbols), period))
        for symbol, exchange in zip(symbols, exchanges):
            # add currency column
            currency = get_value(sql=" select currency from company_profile where symbol = '%s' "%symbol, port=port)
            if currency: currency = currency[0][0]
            else: currency = None
            # add currency column
            country = get_value(sql=" select country from company_profile where symbol = '%s' "%symbol, port=port)
            if country: country = country[0][0]
            else: country = None
            # update table to backfill values
            set_value("""
                update %s 
                set country = '%s', currency = '%s' 
                where symbol = '%s'
            """%(table_name, country, currency, symbol), port=port)
            # run FundamentalTables main script
            ft = FundamentalTables(api_key, session, end_point, symbol, exchange, currency, country, period, limit)
            # create data from end_point
            values = ft.fetch_data()
            if values:
                # create table if not exists for end_point
                create_table(values, table_name, port=port)
                # save data to table
                save(values, table_name, symbol, port=port)
                
        
    except Exception as e:
        logger.error(e)


def mp_main(*arg):
    main(arg)     
        



if __name__ == '__main__':
    args = [
        # (endpoint, period, limit, table_name)
        ('income-statement', 'annual', 30, 'annual_income_statement'),
        ('income-statement', 'quarter', 120, 'quarter_income_statement'),
        ('income-statement-growth', 'annual', 30, 'annual_income_statement_growth'),
        ('income-statement-growth', 'quarter', 120, 'quarter_income_statement_growth'),
        ('balance-sheet-statement', 'annual', 30, 'annual_balance_sheet_statement'),
        ('balance-sheet-statement', 'quarter', 120, 'quarter_balance_sheet_statement'),
        ('balance-sheet-statement-growth', 'annual', 30, 'annual_balance_sheet_statement_growth'),
        ('balance-sheet-statement-growth', 'quarter', 120, 'quarter_balance_sheet_statement_growth'),
        ('cash-flow-statement', 'annual', 30, 'annual_cash_flow_statement'),
        ('cash-flow-statement', 'quarter', 120, 'quarter_cash_flow_statement'),
        ('cash-flow-statement-growth', 'annual', 30, 'annual_cash_flow_statement_growth'),
        ('cash-flow-statement-growth', 'quarter', 120, 'quarter_cash_flow_statement_growth'),
        ('ratios', 'annual', 30, 'annual_ratios'),
        ('ratios', 'quarter', 120, 'quarter_ratios'),
        ('key-metrics', 'annual', 30, 'annual_key_metrics'),
        ('key-metrics', 'quarter', 120, 'quarter_key_metrics')
    ]
    args = [(*arg, conn_params['port']) for arg in args]
    
    threads=[]
    for argument in args:
        t1 = threading.Thread(target=mp_main,args=(argument),daemon=True)
        threads.append(t1)
        threads[-1].start()
        
    for thread in threads:
        thread.join()



fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching symbols and exchanges data
fetching data from income-statement for 9609 companies for period annual
fetching data from balance-sheet-statement for 9738 companies for period quarter
fetching data from income-statement for 9735 companies for period quarter
fetching data from income-statement-growth for 9732 companies for period quarter
fetching data from key-metrics for 9733 companies for period annual
fetching data from balance-sheet-statement for 